The purpose of this notebook is to create a function that takes the imported player list:
- clean to only needed information
- seperate by pither and batter
- merge each on player name to get projections from model
- merge back pitcher and batter data 
- output df for now

Next steps:
- build line up based on salary

In [71]:
# imports
import pandas as pd
import numpy as np

In [72]:
# will need fanduel import
fd = pd.read_csv('../CapStone_Data/FanDuel-MLB-2021 ET-05 ET-06 ET-58417-players-list (2).csv')

In [73]:
# print first 5 rows of fd
fd.head()

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Probable Pitcher,Batting Order,Roster Position
0,58417-52859,P,Jacob,Jacob deGrom,deGrom,61.600000,5.0,12500,NYM@STL,NYM,STL,DTD,Lat,NaN,NaN,0.0,P
1,58417-16956,P,Gerrit,Gerrit Cole,Cole,54.166667,6.0,12300,HOU@NYY,NYY,HOU,NaN,NaN,NaN,Yes,NaN,P
2,58417-82554,P,Shane,Shane Bieber,Bieber,53.428571,7.0,12200,CLE@KC,CLE,KC,NaN,NaN,NaN,NaN,NaN,P
3,58417-5481,P,Max,Max Scherzer,Scherzer,42.166667,6.0,12000,ATL@WSH,WSH,ATL,NaN,NaN,NaN,NaN,NaN,P
4,58417-82604,P,Corbin,Corbin Burnes,Burnes,49.600000,5.0,11100,MIL@PHI,MIL,PHI,IL,Undisclosed,NaN,NaN,0.0,P


only columns needed:
- id - will need this later for template
- Position
- nickname - renamed to Name
- salary
- game
- team
- opponent
- injury indicator
- probable pitcher

In [74]:
fd.columns

Index(['Id', 'Position', 'First Name', 'Nickname', 'Last Name', 'FPPG',
       'Played', 'Salary', 'Game', 'Team', 'Opponent', 'Injury Indicator',
       'Injury Details', 'Tier', 'Probable Pitcher', 'Batting Order',
       'Roster Position'],
      dtype='object')

In [75]:
# clean up fd to match column list above
fd.drop(columns=['First Name', 'Last Name', 'FPPG', 'Played',
                 'Injury Details', 'Tier', 'Batting Order', 'Roster Position'], inplace=True)

Next step is fill nulls in probable pitcher and injury indicator

In [76]:
# filling nulls for probable pitcher
fd['Probable Pitcher'].fillna('No', inplace=True)

In [77]:
# fill nulls for injury indicator
fd['Injury Indicator'].fillna('Healthy', inplace=True)

In [78]:
# review new cleaned df
fd.head()

,Id,Position,Nickname,Salary,Game,Team,Opponent,Injury Indicator,Probable Pitcher
0,58417-52859,P,Jacob deGrom,12500,NYM@STL,NYM,STL,DTD,No
1,58417-16956,P,Gerrit Cole,12300,HOU@NYY,NYY,HOU,Healthy,Yes
2,58417-82554,P,Shane Bieber,12200,CLE@KC,CLE,KC,Healthy,No
3,58417-5481,P,Max Scherzer,12000,ATL@WSH,WSH,ATL,Healthy,No
4,58417-82604,P,Corbin Burnes,11100,MIL@PHI,MIL,PHI,IL,No


In [79]:
fd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 604 entries, 0 to 603
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Id                604 non-null    object
 1   Position          604 non-null    object
 2   Nickname          604 non-null    object
 3   Salary            604 non-null    int64 
 4   Game              604 non-null    object
 5   Team              604 non-null    object
 6   Opponent          604 non-null    object
 7   Injury Indicator  604 non-null    object
 8   Probable Pitcher  604 non-null    object
dtypes: int64(1), object(8)
memory usage: 42.6+ KB


Next step rename Nickname to name

In [80]:
# renaming nickname column
fd.rename(columns={'Nickname': 'Name'}, inplace=True)

Next step filter to only healthy players

In [81]:
# fitler to only healthy players
fd = fd.loc[fd['Injury Indicator'] == 'Healthy']

In [82]:
# review dataframe
fd.head()

,Id,Position,Name,Salary,Game,Team,Opponent,Injury Indicator,Probable Pitcher
1,58417-16956,P,Gerrit Cole,12300,HOU@NYY,NYY,HOU,Healthy,Yes
2,58417-82554,P,Shane Bieber,12200,CLE@KC,CLE,KC,Healthy,No
3,58417-5481,P,Max Scherzer,12000,ATL@WSH,WSH,ATL,Healthy,No
5,58417-13268,P,Danny Duffy,10700,CLE@KC,KC,CLE,Healthy,Yes
7,58417-80582,P,Brandon Woodruff,10100,MIL@PHI,MIL,PHI,Healthy,Yes


In [83]:
fd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 493 entries, 1 to 603
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Id                493 non-null    object
 1   Position          493 non-null    object
 2   Name              493 non-null    object
 3   Salary            493 non-null    int64 
 4   Game              493 non-null    object
 5   Team              493 non-null    object
 6   Opponent          493 non-null    object
 7   Injury Indicator  493 non-null    object
 8   Probable Pitcher  493 non-null    object
dtypes: int64(1), object(8)
memory usage: 38.5+ KB


next step is to split into pithcers and batters

In [84]:
# split using .loc by position and make new dataframe for pitchers
pitchers = fd.loc[fd['Position']=='P']

In [85]:
# split using .loc by position and make new dataframe for batters
batters = fd.loc[fd['Position']!='P']

Now we have cleaned dataframes for each group. Now lets focus on pitchers to get projections added to dataframe.

First step for pitchers is to get the probable pitchers first, we only care about the ones that will start.

In [86]:
# save pitchers df to only starting pitchers
pitchers = pitchers.loc[pitchers['Probable Pitcher']=='Yes']

In [87]:
# review changes
pitchers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15 entries, 1 to 123
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Id                15 non-null     object
 1   Position          15 non-null     object
 2   Name              15 non-null     object
 3   Salary            15 non-null     int64 
 4   Game              15 non-null     object
 5   Team              15 non-null     object
 6   Opponent          15 non-null     object
 7   Injury Indicator  15 non-null     object
 8   Probable Pitcher  15 non-null     object
dtypes: int64(1), object(8)
memory usage: 1.2+ KB


Next step is to combine projections with 2021 stats.
- import testing data with projections from model
- merge the two data frames

In [88]:
# read in pitcher projections
pitcher_proj = pd.read_csv('../Projections/pitcher_projections_2021.csv')

Next step merge.

In [89]:
# merge attempt
pitcher_projections = pitchers.merge(pitcher_proj, how='left', on='Name')

Next step is to drop nulls from this list.  Will need to handle this in previous processes for final product.

In [90]:
pitcher_projections.dropna(inplace=True)

Next step is to clean up and only take what is needed from pitchers so that it can be merged with batters.
Columns needed:
- ID
- Position
- Name
- Salary
- Team
- Opponent
- Proj_FPPG

In [91]:
# overwrite df with only the columns needed
pitcher_projections = pitcher_projections[['Id', 'Position', 'Name', 'Salary', 'Team_x', 'Opponent', 'Proj_FPPG']]

In [92]:
# rename team column
pitcher_projections.rename(columns={'Team_x' : 'Team'}, inplace=True)

Now we have a cleaned and organized df of our pitchers that are starting. Next is the batters.

In [93]:
pitcher_projections

,Id,Position,Name,Salary,Team,Opponent,Proj_FPPG
0,58417-16956,P,Gerrit Cole,12300,NYY,HOU,19.099635
1,58417-13268,P,Danny Duffy,10700,KC,CLE,22.782965
2,58417-80582,P,Brandon Woodruff,10100,MIL,PHI,22.728110
3,58417-16960,P,Zack Wheeler,9100,PHI,MIL,24.214464
4,58417-12864,P,Michael Pineda,8700,MIN,TEX,18.153077
5,58417-13233,P,Nathan Eovaldi,8600,BOS,DET,19.028299
6,58417-52159,P,Lance McCullers Jr.,8400,HOU,NYY,25.259407
7,58417-68518,P,Spencer Turnbull,7800,DET,BOS,14.464556
8,58417-21097,P,Taijuan Walker,7500,NYM,STL,25.634592
9,58417-68654,P,John Gant,7300,STL,NYM,28.366851


---

First step is to merge on projections.

In [94]:
# read in projections file
batter_21 = pd.read_csv('../Projections/batter_projections_2021.csv')

In [95]:
# merge projections with batter df, creating new df
batters_projections = batters.merge(batter_21, how='left', on='Name')

Will have to drop nulls for now, but need to figure out why there is no data for those batters.

In [96]:
# dropping batters with no projections
batters_projections.dropna(inplace=True)

In [97]:
# drop unneeded columns for merge with pitcher, overwrite current df
batters_projections = batters_projections[['Id', 'Position', 'Name', 'Salary', 'Team', 'Opponent','Proj_FPPG']]

Now there are two clean dataframes with the same columns, now we will determine optimal line up.

Steps to get optimal line up:
1. set salary cap
2. sort by fppg

    a. choose pitcher - based on projected fppg; in final product need to incoporate stats with strong coefficient values
    b. choose batters based on remaining salary and highest fppg; in final product need to incoporate stats with strong coefficient values
    
3. create list to append choices to
4. save to one df for output
5. take df and save to templet for export

In [98]:
# set cap for fanduel
salary_cap = 35_000

In [99]:
# sort pitcher by fppg projections
pitcher_projections.sort_values(by='Proj_FPPG', ascending=False, inplace=True, ignore_index=True)

In [100]:
# create a player list to 
lineup = []
lineup.append(pitcher_projections.values[0])

In [101]:
# need to update remaining salary
salary_cap -= pitcher_projections['Salary'][0]

In [102]:
# with updated salary fill remaining roster based on position and highest fppg
# create position list for remaining roster spots
position_list = ['C', '1B', '2B', '3B', 'SS', 'OF', 'OF', 'OF']
# sort batters by FPPG
batters_projections.sort_values(by='Proj_FPPG', ascending=False, inplace=True, ignore_index=True)

In [103]:
# redo 
# create count based on remaining positions
sal_count = 8

# create average salary variable for remaining players
avg_sal = salary_cap/sal_count

# create for loop for each position in list to take highest fppg
for pos in position_list:
    # setting counter to increase if player is already in list
    # this is inside the for loop beacuse it needs to be per position
    counter = 0
    # if salary greater than average move to next player
    for salary in batters_projections.loc[batters_projections['Position'] == pos]['Salary']:
        # test if salary is greater than average if it is increase counter
        if batters_projections.loc[batters_projections['Position'] == pos]['Salary'].values[counter] > avg_sal:
            counter += 1
        else:
            # if less than average add player to list
            lineup.append(batters_projections.loc[batters_projections['Position'] == pos].values[counter])
            # drop player so no duplicates are added
            batters_projections.drop(batters_projections.loc[batters_projections['Position'] == pos].index.values[counter], inplace=True)
            # decrease sal_count
            sal_count -= 1
            # decrease salary cap
            salary_cap -= batters_projections.loc[batters_projections['Position'] == pos]['Salary'].values[counter]
            # create new average salary
            avg_sal = salary_cap/sal_count
            break


<ipython-input-103-595d937b3060>:28: RuntimeWarning: divide by zero encountered in longlong_scalars
  avg_sal = salary_cap/sal_count


In [104]:
df = pd.DataFrame(lineup, columns=['Id', 'Position', 'Name', 'Salary', 'Team', 'Opponent','Proj_FPPG'])
df

,Id,Position,Name,Salary,Team,Opponent,Proj_FPPG
0,58417-68654,P,John Gant,7300,STL,NYM,28.366851
1,58417-79174,C,Jacob Nottingham,2800,MIL,PHI,20.761754
2,58417-52175,1B,Matt Olson,3500,OAK,TOR,8.416243
3,58417-68587,2B,Ozzie Albies,3200,ATL,WSH,7.157615
4,58417-5104,3B,Pablo Sandoval,2000,ATL,WSH,8.426209
5,58417-37982,SS,Xander Bogaerts,3700,BOS,DET,7.656552
6,58417-5763,OF,Nelson Cruz,3900,MIN,TEX,8.714154
7,58417-60643,OF,Aaron Judge,3900,NYY,HOU,8.361619
8,58417-39086,OF,Byron Buxton,4400,MIN,TEX,10.177979


In [105]:
df['Salary'].sum()

34700